In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

/opt/homebrew/anaconda3/envs/main/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [33]:
# check how many of phenocodes are repated with different wording
# this function is used to find the most similar phenocode for each phenocode
def fuzzy_match(dat,column):
    dat['similar_phenocode'] = ''
    dat['score'] = ''
    for index, row in dat.iterrows():
        phenocode = row[column]
        max_score = -1
        max_index = -1
        for index2, row2 in dat.iterrows():
            if index == index2:
                continue
            score = fuzz.ratio(phenocode, row2[column])
            if score > max_score:
                max_score = score
                max_index = index2
        dat.at[index, 'similar_phenocode'] = dat.at[max_index, column]
        dat.at[index, 'score'] = max_score
    return dat

def fuzzy_match_two_df(dat, dat1, column, column1):
    dat['similar_phenocode'] = ''
    dat['score'] = ''
    for index, row in dat.iterrows():
        phenocode = row[column]
        max_score = -1
        max_index = -1
        for index2, row2 in dat1.iterrows():
            if pd.isna(row2[column1]):
                continue
            score = fuzz.ratio(phenocode, row2[column1])
            if score > max_score:
                max_score = score
                max_index = index2
        dat.at[index, 'similar_phenocode'] = dat1.at[max_index, column1]
        dat.at[index, 'score'] = max_score
    return dat

In this section, we aim to get the phenotype hierarchical schema of the PANUKBiobank phenotype manifest. Here are the steps we need to take for each phenotype category:
* trait_type == "icd10" --> use phenocode
* trait_type == "phecode" --> use phecode map to icd10 from here `phecode_icd10.csv` (downloaded from https://phewascatalog.org/phecodes_icd10)
* trait_type == "categorical" & category in "UK Biobank Assessment Centre > Verbal interview > Medical conditions" --> look up `simpleTabulation.csv` (ClassKind == "category") downloaded from https://icd.who.int/browse11/l-m/en

** Note: Two datasets `PHECODE_FINAL.csv` and `CATEGORICAL_MEDICAL_FINAL.csv` are found from the same process we outline here. They were found on a smaller subset of the total UKBiobank data at the start of the project. Though, exact same process was used to find them.

For these two categories, match them with `PRESCRIPTIONS_FINAL.csv` and `CATEGORICAL_MEDICATION_FINAL.csv` first:
* trait_type == "categorical" & category in "UK Biobank Assessment Centre > Verbal interview > Medications" --> find and map from `BNF_Snomed_Mapping_data_20230322.csv` their BNFID (downloaded from https://www.nhsbsa.nhs.uk/prescription-data/understanding-our-data/bnf-snomed-mapping)
* trait_type == "prescriptions" --> find and map from `BNF_Snomed_Mapping_data_20230322.csv` their BNFID (downloaded from https://www.nhsbsa.nhs.uk/prescription-data/understanding-our-data/bnf-snomed-mapping)

Once we have all ICD10 codes, we will map them with ICD11 codes:
* Map ICD10 to ICD11 by using `10To11MapToMultipleCategories.csv` or `10To11MapToOneCategory.csv` (found from https://icd.who.int/browse11/l-m/en)
* Finally use a pckage to get the hieracrchical schema of phenotypes


In [7]:
# load phenotype manifest
pheno_manifest = pd.read_csv("./Data/ukbb_manifest_filtered_phenos.csv")
# load icd10 phecodes
icd10_phecodes = pd.read_csv("./Data/phecode_icd10.csv")
# PheCode to string in icd10_phecodes
icd10_phecodes['PheCode'] = icd10_phecodes['PheCode'].astype(float)
# load simpleTabulation
simpleTabulation = pd.read_csv("./Data/simpleTabulation.csv")
# load bnf mapping data for prescriptions
bnf_mapping = pd.read_csv("./Data/BNF_Snomed_Mapping_data_20230322.csv")
# need to remove duplicated DM+D: Product Description and non NaN values
bnf_mapping = bnf_mapping.drop_duplicates(subset='DM+D: Product Description',keep='last')
bnf_mapping = bnf_mapping.dropna(subset=['DM+D: Product Description'])
# load icd10 to icd11 mapping (multiple categories)
icd10_icd11 = pd.read_csv("./Data/10To11MapToMultipleCategories.csv")
# load phecodes already mapped to icd10
phecode_final = pd.read_csv("./Data/PHECODE_FINAL.csv",sep=';')
# load categoiral medication already mapped to icd10
categorical_medication_icd10 = pd.read_csv("./Data/CATEGORICAL_MEDICATION_FINAL.csv",sep=';')
# load categorical medical already mapped to icd10
categorical_medical_icd10 = pd.read_csv("./Data/CATEGORICAL_MEDICAL_FINAL.csv")
# load prescriptions already mapped to icd10
prescriptions_icd10 = pd.read_csv("./Data/PRESCRIPTIONS_FINAL.csv")

/var/folders/2w/p5yxl8053wbd8p2sw2c_t9hw0000gn/T/ipykernel_43258/216924922.py:8: DtypeWarning: Columns (3,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  simpleTabulation = pd.read_csv("./Data/simpleTabulation.csv")
/var/folders/2w/p5yxl8053wbd8p2sw2c_t9hw0000gn/T/ipykernel_43258/216924922.py:10: DtypeWarning: Columns (7,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  bnf_mapping = pd.read_csv("./Data/BNF_Snomed_Mapping_data_20230322.csv")


In [5]:
# Looking at the trait_type column of the phenotype manifest
pheno_manifest.trait_type.value_counts()

categorical      963
phecode          541
icd10            359
prescriptions    326
continuous       188
biomarkers        24
LOAD               2
Name: trait_type, dtype: int64

In [71]:
# seperate the phenotypes manifest to all categories in trait_type
# continuous
pheno_manifest_continuous = pheno_manifest.query('trait_type == "continuous"')
# biomarkers
pheno_manifest_biomarkers = pheno_manifest.query('trait_type == "biomarkers"')
# categorical
pheno_manifest_categorical = pheno_manifest.query('trait_type == "categorical"')
pheno_manifest_categorical_medical = pheno_manifest.query('trait_type == "categorical" & category == "UK Biobank Assessment Centre > Verbal interview > Medical conditions"')
pheno_manifest_categorical_medication = pheno_manifest.query('trait_type == "categorical" & category == "UK Biobank Assessment Centre > Verbal interview > Medications"')
pheno_manifest_categorical_other = pheno_manifest.query('trait_type == "categorical" & category != "UK Biobank Assessment Centre > Verbal interview > Medical conditions" & category != "UK Biobank Assessment Centre > Verbal interview > Medications"')
pheno_manifest_categorical_operations = pheno_manifest_categorical_other.query('category == "UK Biobank Assessment Centre > Verbal interview > Operations"| category == "Health-related outcomes > Hospital inpatient > Summary Operations"')
pheno_manifest_categorical_admin = pheno_manifest_categorical_other.query('category == "Health-related outcomes > Hospital inpatient > Summary Administration"')
pheno_manifest_categorical_other = pheno_manifest_categorical_other.query('category != "UK Biobank Assessment Centre > Verbal interview > Operations" & category != "Health-related outcomes > Hospital inpatient > Summary Operations" & category != "Health-related outcomes > Hospital inpatient > Summary Administration"')
# phecode
pheno_manifest_phecode = pheno_manifest.query('trait_type == "phecode"')
pheno_manifest_phecode = pheno_manifest_phecode.assign(phenocode = pheno_manifest_phecode['phenocode'].astype(float))
# icd10
pheno_manifest_icd10 = pheno_manifest.query('trait_type == "icd10"')
# prescription
pheno_manifest_prescriptions = pheno_manifest.query('trait_type == "prescriptions"')

In [566]:
print(bnf_mapping.shape)
print(pheno_manifest_phecode.shape)
print(pheno_manifest_categorical.shape)
print(pheno_manifest_categorical_medical.shape)
print(pheno_manifest_categorical_medication.shape)
print(pheno_manifest_categorical_other.shape)
print(pheno_manifest_prescription.shape)


(114978, 11)
(541, 56)
(963, 56)
(120, 56)
(170, 56)
(673, 56)
(326, 56)


#### A) Continous and Biomarkers

In [10]:
# continous and biomarkers do not require ICD10 mapping
pheno_manifest_continuous = (pheno_manifest_continuous
                             .assign(ICD10_code_new = "not_required")
                             .assign(to_remove_dups = 0))
# how about fuzzy matching?
pheno_manifest_continuous = fuzzy_match(pheno_manifest_continuous,'description')
pheno_manifest_biomarkers = (pheno_manifest_biomarkers
                             .assign(ICD10_code_new = "not_required")
                             .assign(to_remove_dups = 0))
# saving
pheno_manifest_continuous.to_csv("./Data/CONTINOUS_UKBB_pheno_manifest.csv",index=False)
pheno_manifest_biomarkers.to_csv("./Data/BIOMARKERS_UKBB_pheno_manifest.csv",index=False)

#### B) ICD10

In [11]:
# mapping pheno_manifest_icd10
pheno_manifest_icd10 = (pheno_manifest_icd10
                        .assign(ICD10_code_new = pheno_manifest_icd10['phenocode'])
                        .assign(to_remove_dups = 0))
# pheno_manifest_icd10 = fuzzy_match(pheno_manifest_icd10,'description')
pheno_manifest_icd10.to_csv("./Data/ICD10_UKBB_pheno_manifest.csv",index=False)

#### C) Phecode

In [629]:
# mapping pheno_manifest_phecode
pheno_manifest_phecode = pheno_manifest_phecode.merge(icd10_phecodes[['PheCode','ICD10']], left_on='phenocode', right_on='PheCode', how='left')
pheno_manifest_phecode['ICD10_code_new'] = pheno_manifest_phecode['ICD10']
pheno_manifest_phecode = pheno_manifest_phecode.drop(columns=['ICD10','PheCode'])

In [630]:
print(pheno_manifest_phecode.shape)
print(len(set(pheno_manifest_phecode['phenocode'])))

(2953, 57)
541


In [631]:
# need to group by phenocode and select one row for each group and replace ICD10_code_new with: <first item of ICD10_code_new + '-' + second item of ICD10_code_new> + ...
aa = (pheno_manifest_phecode
        .groupby('phenocode')['ICD10_code_new']
        .apply(lambda x: '-'.join(x.values.astype(str)))
        .reset_index(name='ICD10_code_new')
        .replace({'ICD10_code_new': 'nan'}, 'not_found'))
pheno_manifest_phecode =pheno_manifest_phecode.drop(columns=['ICD10_code_new']).drop_duplicates()
pheno_manifest_phecode = pheno_manifest_phecode.merge(aa, left_on='phenocode', right_on='phenocode', how='left')

In [632]:
# merging pheno_final with pheno_manifest_phecode to compare ICD10_code_new with AB_ICD10
pheno_manifest_phecode = pheno_manifest_phecode.merge(phecode_final[['phenocode','AB_ICD10']], left_on='phenocode', right_on='phenocode', how='left')
# replace ICD10_code_new with AB_ICD10 if AB_ICD10 is not NaN
pheno_manifest_phecode['ICD10_code_new'] = np.where(pheno_manifest_phecode['AB_ICD10'].notnull(), pheno_manifest_phecode['AB_ICD10'], pheno_manifest_phecode['ICD10_code_new'])
pheno_manifest_phecode = pheno_manifest_phecode.drop(columns=['AB_ICD10'])
pheno_manifest_phecode.head(100)

,trait_type,phenocode,pheno_sex,coding,modifier,description,description_more,coding_description,category,aws_link_tabix,...,md5_hex_tabix,phenocode_annotate_lst,ICD10_code,to_remove,new_pheno_annot,lambda_gc_EUR.y,phenotype_qc_EUR,pops_pass_qc,sldsc_25bin_h2_observed_EUR,ICD10_code_new
0,phecode,8.50,both_sexes,NaN,NaN,Bacterial enteritis,NaN,NaN,infectious diseases,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,b196fd5b07eb2bfb9bb0533e1f688ea6,PH_Bacterial_enteritis_h0.05881_n3887,A04.9,0,PH_Bacterial_enteritis_h0.05881,1.030,h2_z_insignificant,NaN,0.0034,A01.0-A02-A02.0-A02.2-A02.8-A02.9-A03-A03.0-A0...
1,phecode,8.52,both_sexes,NaN,NaN,Intestinal infection due to C. difficile,NaN,NaN,infectious diseases,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,875fa6545d2eee28ee6b9c28a64aecc3,PH_Intestinal_infection_due_to_C._difficile_h0...,A04.7,0,PH_Intestinal_infection_due_to_Cdifficile_h0.....,1.020,h2_z_insignificant,NaN,0.0064,A04
2,phecode,38.10,both_sexes,NaN,NaN,Gram negative septicemia,NaN,NaN,infectious diseases,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,0c3698557971e89c4383ba43be0233e5,PH_Gram_negative_septicemia_h0.07525_n1431,A41.50,0,PH_Gram_negative_septicemia_h0.07525,1.010,h2_z_insignificant,NaN,-0.0005,A41
3,phecode,38.20,both_sexes,NaN,NaN,Gram positive septicemia,NaN,NaN,infectious diseases,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,4bb7e046fcd48530dff29e9372233ba8,PH_Gram_positive_septicemia_h0.16674_n718,A41.50,0,PH_Gram_positive_septicemia_h0.16674,0.984,h2_z_insignificant,NaN,0.0009,A41
4,phecode,41.10,both_sexes,NaN,NaN,Staphylococcus infections,NaN,NaN,infectious diseases,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,5405b18630486cd200b02701a81783fc,PH_Staphylococcus_infections_h0.10007_n4389,A49.01,0,PH_Staphylococcus_infections_h0.10007,1.030,h2_z_insignificant,NaN,0.0008,A49.0-B95.6-B95.7-B95.8-M00.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,phecode,275.50,both_sexes,NaN,NaN,Disorders of calcium/phosphorus metabolism,NaN,NaN,endocrine/metabolic,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,aff3b588b8a7159bb31e0c81ba20b919,PH_Disorders_of_calcium_phosphorus_metabolism_...,E83.30,0,PH_Disorders_of_calcium_phosphorus_metabolism_...,1.040,h2_z_insignificant,NaN,0.0007,E83
96,phecode,275.53,both_sexes,NaN,NaN,Disorders of phosphorus metabolism,NaN,NaN,endocrine/metabolic,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,d496036c32e4e65f02d29c8fb6e600db,PH_Disorders_of_phosphorus_metabolism_h0.07220...,E83.30,0,PH_Disorders_of_phosphorus_metabolism_h0.07220,0.987,h2_z_insignificant,NaN,0.0002,E83
97,phecode,276.00,both_sexes,NaN,NaN,"Disorders of fluid, electrolyte, and acid-base...",NaN,NaN,endocrine/metabolic,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,30e8b5762a6d67efd04ec57c5eb1d8fd,PH_Disorders_of_fluid_electrolyte_and_acidbase...,E87.8,0,PH_Disorders_of_fluid_electrolyte_and_acidbase...,1.020,h2_z_insignificant,NaN,0.0013,not_found
98,phecode,276.10,both_sexes,NaN,NaN,Electrolyte imbalance,NaN,NaN,endocrine/metabolic,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,04e43d19e6a9f162c43990bf34a23712,PH_Electrolyte_imbalance_h0.07603_n7372,E87.8,0,PH_Electrolyte_imbalance_h0.07603,1.050,h2_z_insignificant,NaN,0.0032,E87.8


In [633]:
# count in ICD10_code_new how many have - in the string, the False values will be the ones we need to select and map manually
pheno_manifest_phecode['ICD10_code_new'].str.contains('-').value_counts() 

False    392
True     149
Name: ICD10_code_new, dtype: int64

In [641]:
# how about fuzzy matching?
pheno_manifest_phecode = fuzzy_match(pheno_manifest_phecode,'description')
# pheno_manifest_phecode = fuzzy_match_two_df(pheno_manifest_phecode,pheno_manifest_icd10,'description','description')

In [642]:
# need to manually check the ICD10_code_new column and select the best one according to each phenotype description
pheno_manifest_phecode.to_csv("./Data/PHECODE_UKBB_pheno_manifest.csv", index=False)

#### D1) Categorical Medical Conditions

In [601]:
# identifying a unique row for merging
print(pheno_manifest_categorical_medical.shape)
len(set(pheno_manifest_categorical_medical.coding)) # using coding as unique identifier

(120, 56)


120

In [602]:
# making sure the type of coding in both dataset are the same
pheno_manifest_categorical_medical = pheno_manifest_categorical_medical.assign(coding = pheno_manifest_categorical_medical['coding'].astype(float))
categorical_medical_icd10 = categorical_medical_icd10.assign(coding = categorical_medical_icd10['coding'].astype(float))

In [603]:
# mapping pheno_manifest_phecode
pheno_manifest_categorical_medical = pheno_manifest_categorical_medical.merge(categorical_medical_icd10[['coding','AB_ICD10']], left_on='coding', right_on='coding', how='left')
pheno_manifest_categorical_medical['ICD10_code_new'] = pheno_manifest_categorical_medical['AB_ICD10']
pheno_manifest_categorical_medical = pheno_manifest_categorical_medical.drop(columns=['AB_ICD10'])
pheno_manifest_categorical_medical.head()

,trait_type,phenocode,pheno_sex,coding,modifier,description,description_more,coding_description,category,aws_link_tabix,...,md5_hex_tabix,phenocode_annotate_lst,ICD10_code,to_remove,new_pheno_annot,lambda_gc_EUR.y,phenotype_qc_EUR,pops_pass_qc,sldsc_25bin_h2_observed_EUR,ICD10_code_new
0,categorical,20001,both_sexes,1003.0,NaN,cell can,Code for cancer. If the participant was uncert...,skin cancer,UK Biobank Assessment Centre > Verbal intervie...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,cab334d758bae6b9fa551a91157f803e,CA_Medical_conditions_h0.07432_n1436,Z12.83,0,CA_skin_cancer_h0.07432,1.020,h2_z_insignificant,NaN,0.0018,C44
1,categorical,20001,both_sexes,1022.0,NaN,"Cancer code, self-reported",Code for cancer. If the participant was uncert...,colon cancer/sigmoid cancer,UK Biobank Assessment Centre > Verbal intervie...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,0764cfb72a6ddd1eb20af72dd55b06ba,CA_Medical_conditions_h0.06907_n1520,C18.7,0,CA_colon_cancer_sigmoid_cancer_h0.06907,1.020,h2_z_insignificant,NaN,0.0044,C18
2,categorical,20001,both_sexes,1032.0,NaN,"Cancer code, self-reported",Code for cancer. If the participant was uncert...,brain cancer / primary malignant brain tumour,UK Biobank Assessment Centre > Verbal intervie...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,acbb83137c64be424c0fd4909bea46db,CA_Medical_conditions_h0.08757_n220,C71.7,0,CA_brain_cancer_primary_malignant_brain_tumour...,0.982,h2_z_insignificant,NaN,0.0047,C71
3,categorical,20001,both_sexes,1034.0,NaN,"Cancer code, self-reported",Code for cancer. If the participant was uncert...,kidney/renal cell cancer,UK Biobank Assessment Centre > Verbal intervie...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,0a2c6b6d2d2efce2393d5587492c5547,CA_Medical_conditions_h0.12532_n631,C64.9,0,CA_kidney_renal_cell_cancer_h0.12532,0.999,h2_z_insignificant,NaN,0.0034,C64
4,categorical,20001,both_sexes,1035.0,NaN,"Cancer code, self-reported",Code for cancer. If the participant was uncert...,bladder cancer,UK Biobank Assessment Centre > Verbal intervie...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,7161f2f926bfcbbe2bab06bbfeede613,CA_Medical_conditions_h0.09682_n1118,D09.0,0,CA_bladder_cancer_h0.09682,1.000,h2_z_insignificant,NaN,-0.0016,C67


In [604]:
# get number of nan values in ICD10_code_new
pheno_manifest_categorical_medical['ICD10_code_new'].isnull().sum() 

35

In [605]:
pheno_manifest_categorical_medical = fuzzy_match(pheno_manifest_categorical_medical,'coding_description')

In [608]:
# saving
pheno_manifest_categorical_medical.to_csv("./Data/CATEGORICAL_MEDICAL_UKBB_pheno_manifest.csv", index=False)

#### D2) Categorical Medications

In [594]:
# identifying a unique row for merging
print(pheno_manifest_categorical_medication.shape)
len(set(pheno_manifest_categorical_medication.coding)) # using coding as unique identifier

(170, 56)


170

In [595]:
# making sure the type of coding in both dataset are the same
pheno_manifest_categorical_medication = pheno_manifest_categorical_medication.assign(coding = pheno_manifest_categorical_medication['coding'].astype(float))
categorical_medication_icd10 = categorical_medication_icd10.assign(coding = categorical_medication_icd10['coding'].astype(float))

In [596]:
# mapping pheno_manifest_phecode
pheno_manifest_categorical_medication = pheno_manifest_categorical_medication.merge(categorical_medication_icd10[['coding','AB_ICD10']], left_on='coding', right_on='coding', how='left')
pheno_manifest_categorical_medication['ICD10_code_new'] = pheno_manifest_categorical_medication['AB_ICD10']
pheno_manifest_categorical_medication = pheno_manifest_categorical_medication.drop(columns=['AB_ICD10'])
pheno_manifest_categorical_medication.head()

,trait_type,phenocode,pheno_sex,coding,modifier,description,description_more,coding_description,category,aws_link_tabix,...,md5_hex_tabix,phenocode_annotate_lst,ICD10_code,to_remove,new_pheno_annot,lambda_gc_EUR.y,phenotype_qc_EUR,pops_pass_qc,sldsc_25bin_h2_observed_EUR,ICD10_code_new
0,categorical,20003,both_sexes,1.140852e+09,NaN,Treatment/medication code,Code for treatment Negative codes indicate fre...,gtn 400micrograms spray,UK Biobank Assessment Centre > Verbal intervie...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,fe156e3cbabdfc43ef9912521b6b335c,CA_Medications_h0.15787_n1119,NaN,0,CA_PR_gtn_400micrograms_spray_h0.15787,1.00,h2_z_insignificant,NaN,0.0019,I20_I25
1,categorical,20003,both_sexes,1.140861e+09,NaN,Treatment/medication code,Code for treatment Negative codes indicate fre...,lisinopril,UK Biobank Assessment Centre > Verbal intervie...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,3ca2bb21a0755cf57c3c2c4929dd2209,CA_PR_lisinopril_h0.05444_n13336,NaN,0,CA_PR_lisinopril_h0.05444,1.03,not_EUR_plus_1,NaN,0.0113,NaN
2,categorical,20003,both_sexes,1.140861e+09,NaN,Treatment/medication code,Code for treatment Negative codes indicate fre...,ramipril,UK Biobank Assessment Centre > Verbal intervie...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,bbc4938c425f96f9f83566ff84f414c6,CA_PR_ramipril_h0.06008_n22415,NaN,0,CA_PR_ramipril_h0.06008,1.06,not_EUR_plus_1,NaN,0.0215,NaN
3,categorical,20003,both_sexes,1.140861e+09,NaN,Treatment/medication code,Code for treatment Negative codes indicate fre...,glyceryl trinitrate,UK Biobank Assessment Centre > Verbal intervie...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,fe858c1a6bc117c1a0f97475de80a82a,CA_Medications_h0.08286_n1469,T65.5,0,CA_PR_glyceryl_trinitrate_h0.08286,1.02,h2_z_insignificant,NaN,-0.0020,I20_I25
4,categorical,20003,both_sexes,1.140861e+09,NaN,Treatment/medication code,Code for treatment Negative codes indicate fre...,isosorbide mononitrate,UK Biobank Assessment Centre > Verbal intervie...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,c54799ce964929120b411e66ea40dcc8,CA_PR_isosorbide_mononitrate_h0.06857_n1588,NaN,0,CA_PR_isosorbide_mononitrate_h0.06857,1.03,h2_z_insignificant,NaN,0.0045,NaN


In [589]:
# get number of nan values in ICD10_code_new
pheno_manifest_categorical_medication['ICD10_code_new'].isnull().sum() 

52

In [597]:
# how about a fuzzy match?
pheno_manifest_categorical_medication = fuzzy_match(pheno_manifest_categorical_medication,'coding_description')
# to manually check the top scores to identify any duplicates

In [598]:
# saving
pheno_manifest_categorical_medication.to_csv("./Data/CATEGORICAL_MEDICATION_UKBB_pheno_manifest.csv", index=False)

#### D3) Categorical Other

In [72]:
pheno_manifest_categorical_operations = pheno_manifest_categorical_operations.assign(ICD10_code_new='not_found')
pheno_manifest_categorical_admin = pheno_manifest_categorical_admin.assign(ICD10_code_new='not_found')
pheno_manifest_categorical_other = pheno_manifest_categorical_other.assign(ICD10_code_new='not_found')

In [73]:
# fuzzy matching each of the categorical operations
pheno_manifest_categorical_operations = fuzzy_match(pheno_manifest_categorical_operations,'coding_description')
pheno_manifest_categorical_admin = fuzzy_match(pheno_manifest_categorical_admin,'coding_description')
pheno_manifest_categorical_other = fuzzy_match(pheno_manifest_categorical_other,'description')

# to manually check the top scores to identify any duplicates
pheno_manifest_categorical_operations.to_csv("./Data/CATEGORICAL_OPERATIONS_UKBB_pheno_manifest.csv", index=False)
pheno_manifest_categorical_admin.to_csv("./Data/CATEGORICAL_ADMIN_UKBB_pheno_manifest.csv", index=False)
pheno_manifest_categorical_other.to_csv("./Data/CATEGORICAL_OTHER_UKBB_pheno_manifest.csv", index=False)

#### E) Prescriptions

In [558]:
# identifying a unique row for merging
print(pheno_manifest_prescriptions.shape)
len(set(pheno_manifest_prescriptions.phenocode)) # using coding as unique identifier

(326, 56)


326

In [559]:
# check if all the phenocodes match 
print(prescriptions_icd10.shape)
pheno_manifest_prescriptions['phenocode'].isin(prescriptions_icd10['phenocode']).value_counts()

(65, 58)


False    261
True      65
Name: phenocode, dtype: int64

In [560]:
# mapping pheno_manifest_phecode
pheno_manifest_prescriptions = pheno_manifest_prescriptions.merge(prescriptions_icd10[['phenocode','AB_ICD10']], left_on='phenocode', right_on='phenocode', how='left')
pheno_manifest_prescriptions['ICD10_code_new'] = pheno_manifest_prescriptions['AB_ICD10']
pheno_manifest_prescriptions = pheno_manifest_prescriptions.drop(columns=['AB_ICD10'])
pheno_manifest_prescriptions.head()

,trait_type,phenocode,pheno_sex,coding,modifier,description,description_more,coding_description,category,aws_link_tabix,...,md5_hex_tabix,phenocode_annotate_lst,ICD10_code,to_remove,new_pheno_annot,lambda_gc_EUR.y,phenotype_qc_EUR,pops_pass_qc,sldsc_25bin_h2_observed_EUR,ICD10_code_new
0,prescriptions,5-alpha reductase inhibitor|BPH|benign prostat...,both_sexes,NaN,NaN,NaN,NaN,NaN,NaN,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,ee7727601807fdcde15a2032e09942c4,PR_5-alpha_reductase_inhibitor_BPH_benign_pros...,NaN,0,PR_5-alpha_reductase_inhibitor_BPH_benign_pros...,1.05,h2_z_insignificant,NaN,0.0078,NaN
1,prescriptions,abortive anti-migraine agent|selective seroton...,both_sexes,NaN,NaN,NaN,NaN,NaN,NaN,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,c046d286343a9ea7d036825005127a29,PR_abortive_antimigraine_agent_selective_serot...,NaN,0,PR_abortive_antimigraine_agent_selective_serot...,1.06,h2_z_insignificant,NaN,0.0048,NaN
2,prescriptions,ACE inhibitor|anti-hypertensive,both_sexes,NaN,NaN,NaN,NaN,NaN,NaN,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,2af8762f6869857d36a5fcbed81a100e,PR_ACE_inhibitor_antihypertensive_h0.07074_n44865,T46.4X1A,0,PR_ACE_inhibitor_antihypertensive_h0.07074,1.16,not_EUR_plus_1,NaN,0.0263,NaN
3,prescriptions,acyclovir,both_sexes,NaN,NaN,NaN,NaN,NaN,Antiviral,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,efacdcec0425e8a2ed784f30eef280c4,PR_acyclovir_h0.05562_n8213,NaN,0,PR_acyclovir_h0.05562,1.05,h2_z_insignificant,NaN,0.0038,NaN
4,prescriptions,aerochamber,both_sexes,NaN,NaN,NaN,NaN,NaN,delivery system,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,fc159ada1371e09e1350880dd7ff113b,PR_aerochamber_h0.09816_n9348,NaN,0,PR_aerochamber_h0.09816,1.06,h2_z_insignificant,NaN,0.0076,NaN


In [537]:
# get number of nan values in ICD10_code_new
pheno_manifest_prescriptions['ICD10_code_new'].isnull().sum() 

262

In [571]:
# how about a fuzzy match?
pheno_manifest_prescriptions = fuzzy_match(pheno_manifest_prescriptions,'phenocode')
# to manually check the top scores to identify any duplicates

In [564]:
# saving
pheno_manifest_prescriptions.to_csv("./Data/PRESCRIPTIONS_UKBB_pheno_manifest.csv", index=False)

#### F) Combining prescrptions and categorical medications

In [13]:
# reading in the new data from modified_pheno_categories_self_compared folder which we manually checked and updated
pheno_manifest_categorical_medication = pd.read_csv("./Data/modified_pheno_categories_self_compared/CATEGORICAL_MEDICATION_UKBB_pheno_manifest.csv")
pheno_manifest_prescriptions = pd.read_csv("./Data/modified_pheno_categories_self_compared/PRESCRIPTIONS_UKBB_pheno_manifest.csv")

In [15]:
print(pheno_manifest_categorical_medication.shape)
print(pheno_manifest_prescriptions.shape)

(172, 60)
(327, 60)


In [18]:
# before concatinating, we need to make sure the description columns are the same
# need to replace coding_description with phenocode in pheno_manifest_prescriptions
pheno_manifest_prescriptions['coding_description'] = pheno_manifest_prescriptions['phenocode']
# concatinating the two dataframe into one:
pheno_manifest_categorical_medication_prescriptions = pd.concat([pheno_manifest_categorical_medication, pheno_manifest_prescriptions], axis=0)
print(pheno_manifest_categorical_medication_prescriptions.shape)

(499, 60)


In [19]:
# checking if the new data has any empty values in coding_description
pheno_manifest_categorical_medication_prescriptions['coding_description'].isnull().sum()

3

In [21]:
# there are three empty ones, which ones?
pheno_manifest_categorical_medication_prescriptions[pheno_manifest_categorical_medication_prescriptions['coding_description'].isnull()]

,trait_type,phenocode,pheno_sex,coding,modifier,description,description_more,coding_description,category,aws_link_tabix,...,to_remove,new_pheno_annot,lambda_gc_EUR.y,phenotype_qc_EUR,pops_pass_qc,sldsc_25bin_h2_observed_EUR,ICD10_code_new,similar_phenocode,score,to_remove_dups
170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0


In [31]:
# let's remove these rows
pheno_manifest_categorical_medication_prescriptions = pheno_manifest_categorical_medication_prescriptions[pheno_manifest_categorical_medication_prescriptions['coding_description'].notnull()]
# reindexing
pheno_manifest_categorical_medication_prescriptions = pheno_manifest_categorical_medication_prescriptions.reset_index(drop=True)

In [ ]:
# how about a fuzzy match?
pheno_manifest_categorical_medication_prescriptions = fuzzy_match(pheno_manifest_categorical_medication_prescriptions,'coding_description')
# to manually check the top scores to identify any duplicates

In [34]:
# saving the file
pheno_manifest_categorical_medication_prescriptions.to_csv("./Data/CATEGORICAL_MEDICATION_PRESCRIPTIONS_UKBB_pheno_manifest.csv", index=False)

##### F1) Adding BNI codings

In [ ]:
pheno_manifest_categorical_medication_prescriptions = pd.read_csv("./Data/modified_pheno_categories_self_compared/CATEGORICAL_MEDICATION_PRESCRIPTIONS_UKBB_pheno_manifest.csv")

In [35]:
bnf_mapping.shape

(114978, 11)

In [36]:
print(len(set(bnf_mapping['DM+D: Product Description']))) # this column has all unique values

114978


In [37]:
# here we want to identify closest descriptions of the prescriptions to the ones in bnf_mapping 'DM+D: Product Description' and 'coding_description'
pheno_manifest_categorical_medication_prescriptions = fuzzy_match_two_df(pheno_manifest_categorical_medication_prescriptions, bnf_mapping, 'coding_description', 'DM+D: Product Description')

In [40]:
pheno_manifest_categorical_medication_prescriptions.shape

(496, 60)

In [53]:
# before saving pheno_manifest_categorical_medication_prescriptions, we need to extract 'BNF Code' and 'SNOMED Code' from the bnf_mapping dataframe
pheno_manifest_categorical_medication_prescriptions = pheno_manifest_categorical_medication_prescriptions.merge(bnf_mapping[['DM+D: Product Description','BNF Code','SNOMED Code']], left_on='similar_phenocode', right_on='DM+D: Product Description', how='left')

In [56]:
# additionally, we need to get the presumed disease for each prescription from CATEGORICAL_MEDICATION_FINAL and PRESCRIPTIONS_FINAL
pheno_manifest_categorical_medication_prescriptions1 = pheno_manifest_categorical_medication_prescriptions.merge(categorical_medication_icd10[['coding_description','AB_PRESUMED.DIAGNOSIS']], left_on='coding_description', right_on='coding_description', how='left')
pheno_manifest_categorical_medication_prescriptions1 = pheno_manifest_categorical_medication_prescriptions1.merge(prescriptions_icd10[['phenocode','AB_PRESUMED.DIAGNOSIS']], left_on='phenocode', right_on='phenocode', how='left')
# merging AB_PRESUMED.DIAGNOSIS_x and AB_PRESUMED.DIAGNOSIS_y
pheno_manifest_categorical_medication_prescriptions1['AB_PRESUMED.DIAGNOSIS'] = pheno_manifest_categorical_medication_prescriptions1['AB_PRESUMED.DIAGNOSIS_x'].fillna(pheno_manifest_categorical_medication_prescriptions1['AB_PRESUMED.DIAGNOSIS_y'])

In [51]:
pheno_manifest_categorical_medication_prescriptions1.shape

(496, 65)

In [57]:
pheno_manifest_categorical_medication_prescriptions1.to_csv("./Data/CATEGORICAL_MEDICATION_PRESCRIPTIONS_UKBB_pheno_manifest.csv", index=False)

#### H) Combining all medical phenotypes (after manual selection)

In [79]:
# reading CATEGORICAL_MEDICAL_UKBB_pheno_manifest.csv from modified_pheno_categories_self_compared folder
pheno_manifest_categorical_medical = pd.read_csv("./Data/modified_pheno_categories_self_compared/CATEGORICAL_MEDICAL_UKBB_pheno_manifest.csv")
# reading PHECODE_UKBB_pheno_manifest.csv from modified_pheno_categories_self_compared folder
pheno_manifest_phecode = pd.read_csv("./Data/modified_pheno_categories_self_compared/PHECODE_UKBB_pheno_manifest.csv")
# reading ICD10_UKBB_phenotype_manifest.csv from modified_pheno_categories_self_compared folder
pheno_manifest_icd10 = pd.read_csv("./Data/modified_pheno_categories_self_compared/ICD10_UKBB_pheno_manifest.csv")

In [82]:
pheno_manifest_categorical_medical.head() # coding_description

,trait_type,phenocode,pheno_sex,coding,modifier,description,description_more,coding_description,category,aws_link_tabix,...,phenocode_annotate_lst,ICD10_code,to_remove,new_pheno_annot,lambda_gc_EUR.y,phenotype_qc_EUR,pops_pass_qc,sldsc_25bin_h2_observed_EUR,ICD10_code_new,to_remove_dups
0,categorical,20002,both_sexes,1222,NaN,"Non-cancer illness code, self-reported",Code for non-cancer illness. If the participan...,type 1 diabetes,UK Biobank Assessment Centre > Verbal intervie...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,CA_Medical_conditions_h0.14695_n420,E10.618,0,CA_type_1_diabetes_h0.14695,1.02,h2_z_insignificant,NaN,0.0019,E10,0
1,categorical,20002,both_sexes,1223,NaN,"Non-cancer illness code, self-reported",Code for non-cancer illness. If the participan...,type 2 diabetes,UK Biobank Assessment Centre > Verbal intervie...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,CA_type_2_diabetes_h0.15122_n3260,E11.618,0,CA_type_2_diabetes_h0.15122,1.03,PASS,"AFR,EUR",0.0104,E11,0
2,categorical,20001,both_sexes,1061,NaN,"Cancer code, self-reported",Code for cancer. If the participant was uncert...,basal cell carcinoma,UK Biobank Assessment Centre > Verbal intervie...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,CA_Medical_conditions_h0.11939_n4278,C44.91,0,CA_basal_cell_carcinoma_h0.11939,1.06,not_EUR_plus_1,NaN,0.0096,C44,1
3,categorical,20001,both_sexes,1062,NaN,"Cancer code, self-reported",Code for cancer. If the participant was uncert...,squamous cell carcinoma,UK Biobank Assessment Centre > Verbal intervie...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,CA_Medical_conditions_h0.08530_n564,C44.321,0,CA_squamous_cell_carcinoma_h0.08530,1.01,h2_z_insignificant,NaN,0.0024,C44,0
4,categorical,20002,both_sexes,1201,NaN,"Non-cancer illness code, self-reported",Code for non-cancer illness. If the participan...,bladder problem (not cancer),UK Biobank Assessment Centre > Verbal intervie...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,CA_Medical_conditions_h0.06705_n2857,D09.0,0,CA_bladder_problem_not_cancer_h0.06705,1.05,h2_z_insignificant,NaN,0.0035,N39,0


In [83]:
pheno_manifest_phecode.head() # description

,trait_type,phenocode,pheno_sex,coding,modifier,description,description_more,coding_description,category,aws_link_tabix,...,phenocode_annotate_lst,ICD10_code,to_remove,new_pheno_annot,lambda_gc_EUR.y,phenotype_qc_EUR,pops_pass_qc,sldsc_25bin_h2_observed_EUR,ICD10_code_new,to_remove_dups
0,phecode,250.13,both_sexes,NaN,NaN,Type 1 diabetes with ophthalmic manifestations,NaN,NaN,endocrine/metabolic,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,PH_Type_1_diabetes_with_ophthalmic_manifestati...,E10.39,0,PH_Type_1_diabetes_with_ophthalmic_manifestati...,1.02,h2_z_insignificant,NaN,0.0042,E10,0
1,phecode,250.23,both_sexes,NaN,NaN,Type 2 diabetes with ophthalmic manifestations,NaN,NaN,endocrine/metabolic,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,PH_Type_2_diabetes_with_ophthalmic_manifestati...,E11.39,0,PH_Type_2_diabetes_with_ophthalmic_manifestati...,1.04,h2_z_insignificant,NaN,0.0095,E11-E11.0-E11.6-E11.7-E11.8-E11.9-E13-E13.5-E1...,0
2,phecode,300.00,both_sexes,NaN,NaN,Anxiety disorders,NaN,NaN,mental disorders,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,PH_Anxiety_disorders_h0.05015_n11860,F40.8,0,PH_Anxiety_disorders_h0.05015,1.06,PASS,"CSA,EUR",0.0136,F48.0-F48.8-F48.9-F99-R45.2-R45.5,0
3,phecode,300.10,both_sexes,NaN,NaN,Anxiety disorder,NaN,NaN,mental disorders,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,PH_Anxiety_disorder_h0.05160_n11002,F40.8,0,PH_Anxiety_disorder_h0.05160,1.08,PASS,"CSA,EUR",0.0133,F06.4-F41.2-F41.3-F41.8-F41.9,0
4,phecode,721.10,both_sexes,NaN,NaN,Spondylosis without myelopathy,NaN,NaN,musculoskeletal,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,PH_Spondylosis_without_myelopathy_h0.08618_n7632,M47.819,0,PH_Spondylosis_without_myelopathy_h0.08618,1.06,h2_z_insignificant,NaN,0.0055,M47.2-M47.8,0


In [84]:
pheno_manifest_icd10.head() # description

,trait_type,phenocode,pheno_sex,coding,modifier,description,description_more,coding_description,category,aws_link_tabix,...,phenocode_annotate_lst,ICD10_code,to_remove,new_pheno_annot,lambda_gc_EUR.y,phenotype_qc_EUR,pops_pass_qc,sldsc_25bin_h2_observed_EUR,ICD10_code_new,to_remove_dups
0,icd10,A04,both_sexes,NaN,NaN,A04 Other bacterial intestinal infections,truncated: true,NaN,Chapter I Certain infectious and parasitic dis...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,IC_Other_bacterial_intestinal_infections_h0.06...,A04,0,IC_Other_bacterial_intestinal_infections_h0.06490,1.020,h2_z_insignificant,NaN,0.003300,A04,0
1,icd10,A49,both_sexes,NaN,NaN,A49 Bacterial infection of unspecified site,truncated: true,NaN,Chapter I Certain infectious and parasitic dis...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,IC_Bacterial_infection_of_unspecified_site_h0....,A49,0,IC_Bacterial_infection_of_unspecified_site_h0....,0.996,h2_z_insignificant,NaN,0.003500,A49,0
2,icd10,B00,both_sexes,NaN,NaN,B00 Herpesviral [herpes simplex] infections,truncated: true,NaN,Chapter I Certain infectious and parasitic dis...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,IC_Herpesviral_[herpes_simplex]_infections_h0....,B00,0,IC_Herpesviral_herpes_simplex_infections_h0.13719,0.984,h2_z_insignificant,NaN,0.003200,B00,0
3,icd10,B02,both_sexes,NaN,NaN,B02 Zoster [herpes zoster],truncated: true,NaN,Chapter I Certain infectious and parasitic dis...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,IC_Zoster_[herpes_zoster]_h0.12765_n729,B02,0,IC_Zoster_herpes_zoster_h0.12765,1.010,h2_z_insignificant,NaN,0.005100,B02,0
4,icd10,B35,both_sexes,NaN,NaN,B35 Dermatophytosis,truncated: true,NaN,Chapter I Certain infectious and parasitic dis...,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,IC_Dermatophytosis_h0.11636_n356,B35,0,IC_Dermatophytosis_h0.11636,1.000,h2_z_insignificant,NaN,0.000053,B35,0


In [91]:
# checking if all values in coding_description are NaN
print(pheno_manifest_phecode.shape)
print(pheno_manifest_phecode['coding_description'].isnull().sum())
print(pheno_manifest_icd10.shape)
print(pheno_manifest_icd10['coding_description'].isnull().sum())

(541, 58)
541
(359, 58)
359


In [92]:
# before combining we need to copy description to the coding_description column in pheno_manifest_phecode and pheno_manifest_icd10:
pheno_manifest_phecode['coding_description'] = pheno_manifest_phecode['description']
pheno_manifest_icd10['coding_description'] = pheno_manifest_icd10['description']

In [96]:
# now we combine theses three dataframes into one
pheno_manifest_categorical_medical_phecode_icd10 = pd.concat([pheno_manifest_categorical_medical, pheno_manifest_phecode, pheno_manifest_icd10], axis=0)
# reindexing
pheno_manifest_categorical_medical_phecode_icd10 = pheno_manifest_categorical_medical_phecode_icd10.reset_index(drop=True)

In [94]:
pheno_manifest_categorical_medical_phecode_icd10.shape

(1020, 58)

In [97]:
# how about fuzzy match?
pheno_manifest_categorical_medical_phecode_icd10 = fuzzy_match(pheno_manifest_categorical_medical_phecode_icd10,'coding_description')

In [98]:
# saving the file
pheno_manifest_categorical_medical_phecode_icd10.to_csv("./Data/CATEGORICAL_MEDICAL_PHECODE_ICD10_UKBB_pheno_manifest.csv", index=False)

In [100]:
pheno_manifest_categorical_medical_phecode_icd10.query("phenocode==38.1")

,trait_type,phenocode,pheno_sex,coding,modifier,description,description_more,coding_description,category,aws_link_tabix,...,to_remove,new_pheno_annot,lambda_gc_EUR.y,phenotype_qc_EUR,pops_pass_qc,sldsc_25bin_h2_observed_EUR,ICD10_code_new,to_remove_dups,similar_phenocode,score
168,phecode,38.1,both_sexes,NaN,NaN,Gram negative septicemia,NaN,Gram negative septicemia,infectious diseases,https://pan-ukb-us-east-1.s3.amazonaws.com/sum...,...,0,PH_Gram_negative_septicemia_h0.07525,1.01,h2_z_insignificant,NaN,-0.0005,A41,0,Gram positive septicemia,83


# Combining all phenotypes
Now that we have properly cleaned and selected all phenotypes, we can combine them into one dataframe.

In [22]:
import os
# read and concatane all the data avilable in  ./Data/ukbb_manifest_filtered_phenos/ folder
path = "./Data/ukbb_manifest_filtered_phenos/"
files = os.listdir(path)
# reading all the files
ukbb_manifest_filtered_phenos = pd.DataFrame()
for file in files:
    ukbb_manifest_filtered_phenos = pd.concat([ukbb_manifest_filtered_phenos, pd.read_csv(path+file)], axis=0)

ukbb_manifest_filtered_phenos.shape

(2401, 58)

In [23]:
# get two LOAD rows of pheno_manifest data
pheno_LOAD_ext = pheno_manifest.query("trait_type=='LOAD'")
# add these rows to ukbb_manifest_filtered_phenos
# first, we need to add two columns ('ICD10_code_new','to_remove_dups') to pheno_LOAD_ext
pheno_LOAD_ext['ICD10_code_new'] = 'F00'
pheno_LOAD_ext['to_remove_dups'] = 0


/var/folders/2w/p5yxl8053wbd8p2sw2c_t9hw0000gn/T/ipykernel_43258/1588609275.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pheno_LOAD_ext['ICD10_code_new'] = 'F00'
/var/folders/2w/p5yxl8053wbd8p2sw2c_t9hw0000gn/T/ipykernel_43258/1588609275.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pheno_LOAD_ext['to_remove_dups'] = 0


In [24]:
# add these rows to ukbb_manifest_filtered_phenos
ukbb_manifest_filtered_phenos = pd.concat([ukbb_manifest_filtered_phenos, pheno_LOAD_ext], axis=0)

In [25]:
ukbb_manifest_filtered_phenos.to_remove_dups.value_counts()

0    1621
1     782
Name: to_remove_dups, dtype: int64

In [26]:
# saving the new ukbb_manifest_filtered_phenos
ukbb_manifest_filtered_phenos.to_csv("./Data/ukbb_manifest_filtered_phenos/ukbb_manifest_filtered_phenos.csv", index=False)